In [1]:
#!pip install python-dotenv decorator==4.3 networkx
#!pip install pycoingecko
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import pprint


%matplotlib inline

In [2]:
load_dotenv('../.env')

True

In [3]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [4]:
coin_market = cg.get_coins_markets(vs_currency='usd')
coin_market_df= pd.DataFrame(coin_market, columns=['id', 'current_price', 'high_24h' , 'low_24h'])
coin_market_df.set_index('id', inplace=True)
coin_market_df

,current_price,high_24h,low_24h
id,,,
bitcoin,57658.000000,57873.000000,54409.000000
ethereum,3596.570000,3604.230000,3427.330000
binancecoin,465.620000,467.920000,439.400000
cardano,2.190000,2.190000,2.090000
tether,1.000000,1.010000,0.993679
...,...,...,...
bitclout,114.270000,120.970000,114.040000
zilliqa,0.094147,0.094230,0.090157
telcoin,0.020340,0.022063,0.017485


In [5]:
btc_history = cg.get_coin_market_chart_by_id(id= 'bitcoin', vs_currency = 'usd', days = 'max')
btc_history_df = pd.DataFrame(btc_history)

In [6]:
def extract_volume(x):
    return x[0]
def extract_price(x):
    return x[1]

btc_history_df["volume"] = btc_history_df['prices'].apply(extract_volume)
btc_history_df["price"] = btc_history_df['prices'].apply(extract_price)
btc_history_df["market_cap"] = btc_history_df['market_caps'].apply(extract_price)
btc_history_df["total_volume"] = btc_history_df['total_volumes'].apply(extract_price)

In [7]:
btc_history_df = btc_history_df[['volume', 'price',
       'market_cap', 'total_volume']]
btc_history_df["date"] = pd.to_datetime(btc_history_df["volume"], unit="ms")
btc_history_df.set_index('date', inplace=True)
btc_history_df.drop(columns='volume', inplace=True)
btc_history_df.head()

,price,market_cap,total_volume
date,,,
2013-04-28,135.30,1.500518e+09,0.0
2013-04-29,141.96,1.575032e+09,0.0
2013-04-30,135.30,1.501657e+09,0.0
2013-05-01,117.00,1.298952e+09,0.0
2013-05-02,103.43,1.148668e+09,0.0


In [8]:
eth_history = cg.get_coin_market_chart_by_id(id= 'ethereum', vs_currency = 'usd', days = 'max')
eth_history_df = pd.DataFrame(eth_history)


In [9]:
eth_history_df["volume"] = eth_history_df['prices'].apply(extract_volume)
eth_history_df["price"] = eth_history_df['prices'].apply(extract_price)
eth_history_df["market_cap"] = eth_history_df['market_caps'].apply(extract_price)
eth_history_df["total_volume"] = eth_history_df['total_volumes'].apply(extract_price)

In [10]:
eth_history_df = eth_history_df[['volume', 'price',
       'market_cap', 'total_volume']]
eth_history_df["date"] = pd.to_datetime(eth_history_df["volume"], unit="ms")
eth_history_df.set_index('date', inplace=True)
eth_history_df.drop(columns='volume', inplace=True)
eth_history_df.head()

,price,market_cap,total_volume
date,,,
2015-08-07,2.831620,0.000000e+00,9.062200e+04
2015-08-08,1.330750,8.033948e+07,3.680700e+05
2015-08-10,0.687586,4.155631e+07,4.004641e+05
2015-08-11,1.067379,6.453901e+07,1.518998e+06
2015-08-12,1.256613,7.601326e+07,2.073893e+06


In [11]:
lte_history = cg.get_coin_market_chart_by_id(id= 'litecoin', vs_currency = 'usd', days = 'max')
lte_history_df = pd.DataFrame(lte_history)

In [12]:
lte_history_df["volume"] = lte_history_df['prices'].apply(extract_volume)
lte_history_df["price"] = lte_history_df['prices'].apply(extract_price)
lte_history_df["market_cap"] = lte_history_df['market_caps'].apply(extract_price)
lte_history_df["total_volume"] = lte_history_df['total_volumes'].apply(extract_price)

In [13]:
lte_history_df = lte_history_df[['volume', 'price',
       'market_cap', 'total_volume']]
lte_history_df["date"] = pd.to_datetime(lte_history_df["volume"], unit="ms")
lte_history_df.set_index('date', inplace=True)
lte_history_df.drop(columns='volume', inplace=True)
lte_history_df.head()

,price,market_cap,total_volume
date,,,
2013-04-28,4.29983,73773387.0,0.0
2013-04-29,4.35940,74936909.0,0.0
2013-04-30,4.18295,72037636.0,0.0
2013-05-01,3.64914,62957992.0,0.0
2013-05-02,3.38879,58565340.0,0.0


In [14]:
link_history = cg.get_coin_market_chart_by_id(id= 'chainlink', vs_currency = 'usd', days = 'max')
link_history_df = pd.DataFrame(link_history)

In [15]:
link_history_df["volume"] = link_history_df['prices'].apply(extract_volume)
link_history_df["price"] = link_history_df['prices'].apply(extract_price)
link_history_df["market_cap"] = link_history_df['market_caps'].apply(extract_price)
link_history_df["total_volume"] = link_history_df['total_volumes'].apply(extract_price)

In [16]:
link_history_df = link_history_df[['volume', 'price',
       'market_cap', 'total_volume']]
link_history_df["date"] = pd.to_datetime(link_history_df["volume"], unit="ms")
link_history_df.set_index('date', inplace=True)
link_history_df.drop(columns='volume', inplace=True)
link_history_df.head()

,price,market_cap,total_volume
date,,,
2017-11-09,0.225377,7.888183e+07,3.190026e+06
2017-11-10,0.189036,6.616275e+07,2.121175e+06
2017-11-11,0.181223,6.342793e+07,2.229298e+06
2017-11-12,0.170013,5.950462e+07,5.361146e+06
2017-11-13,0.180691,6.324186e+07,2.942234e+06


In [17]:
ada_history = cg.get_coin_market_chart_by_id(id= 'cardano', vs_currency = 'usd', days = 'max')
ada_history_df = pd.DataFrame(ada_history)

In [18]:
ada_history_df["volume"] = ada_history_df['prices'].apply(extract_volume)
ada_history_df["price"] = ada_history_df['prices'].apply(extract_price)
ada_history_df["market_cap"] = ada_history_df['market_caps'].apply(extract_price)
ada_history_df["total_volume"] = ada_history_df['total_volumes'].apply(extract_price)

In [19]:
ada_history_df = ada_history_df[['volume', 'price',
       'market_cap', 'total_volume']]
ada_history_df["date"] = pd.to_datetime(ada_history_df["volume"], unit="ms")
ada_history_df.set_index('date', inplace=True)
ada_history_df.drop(columns='volume', inplace=True)
ada_history_df.head()

,price,market_cap,total_volume
date,,,
2017-10-18,0.026845,6.960214e+08,2.351678e+06
2017-10-19,0.026830,6.956220e+08,2.815156e+06
2017-10-20,0.030300,7.855800e+08,8.883473e+06
2017-10-21,0.028588,7.412021e+08,5.308857e+06
2017-10-22,0.027796,7.206698e+08,2.901876e+06


In [20]:
to_merge_dict = {
    'BTC': btc_history_df , 'ETH': eth_history_df,
    'LTE':lte_history_df,'LINK':link_history_df, 'ADA':ada_history_df }

In [21]:
#Coin Merged DataFrame
merged_coin_df = pd.concat(to_merge_dict.values(), keys=to_merge_dict.keys(), axis=1, join='inner')
merged_coin_df.head()

BTC                                     ETH                \
                  price    market_cap  total_volume       price    market_cap   
date                                                                            
2017-11-09  7186.292775  1.198168e+11  2.330694e+09  322.329520  3.082374e+10   
2017-11-10  6656.839694  1.109994e+11  3.886426e+09  301.354933  2.882416e+10   
2017-11-11  6365.397852  1.061461e+11  3.148370e+09  316.258342  3.025603e+10   
2017-11-12  5866.513129  9.783217e+10  6.001018e+09  314.166128  3.006218e+10   
2017-11-13  6423.091402  1.071254e+11  4.636292e+09  315.343956  3.018143e+10   

                                LTE                                  LINK  \
            total_volume      price    market_cap  total_volume     price   
date                                                                        
2017-11-09  6.918154e+08  64.537577  3.469079e+09  2.345449e+08  0.225377   
2017-11-10  6.505058e+08  59.371318  3.192341e+09  2.366416e+08  0.189036   
2017-11-11  6.186761e+08  62.612170  3.367560e+09  2.256785e+08  0.181223   
2017-11-12  1.172944e+09  58.916112  3.169837e+09  2.526936e+08  0.170013   
2017-11-13  7.833661e+08  61.163042  3.291569e+09  1.543181e+08  0.180691   

                                             ADA                              
              market_cap  total_volume     price    market_cap  total_volume  
date                                                                          
2017-11-09  7.888183e+07  3.190026e+06  0.032769  8.496170e+08  2.062814e+07  
2017-11-10  6.616275e+07  2.121175e+06  0.027293  7.076286e+08  6.563684e+06  
2017-11-11  6.342793e+07  2.229298e+06  0.027499  7.129560e+08  5.632138e+06  
2017-11-12  5.950462e+07  5.361146e+06  0.024522  6.357746e+08  7.550745e+06  
2017-11-13  6.324186e+07  2.942234e+06  0.025114  6.511399e+08  4.284486e+06

In [22]:
merged_coin_df.to_json('./Resources/crypto_data.json')

In [23]:
#Alpaca Keys
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [24]:
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [25]:
aapl_data = api.get_barset('AAPL',timeframe='1D', limit=1000,).df
aapl_data.tail()

AAPL                                      
                             open      high       low   close    volume
time                                                                   
2021-10-07 00:00:00-04:00  143.00  144.2150  142.7200  143.28  54652459
2021-10-08 00:00:00-04:00  144.04  144.1781  142.5600  143.01  48459122
2021-10-11 00:00:00-04:00  142.30  144.8100  141.8100  142.80  55706039
2021-10-12 00:00:00-04:00  143.23  143.2500  141.0401  141.46  57465598
2021-10-13 00:00:00-04:00  141.20  141.4000  139.2000  140.96  66616125

In [26]:
amzn_data = api.get_barset('AMZN',timeframe='1D', limit=1000,).df
amzn_data.head()

AMZN                                     
                              open     high      low    close    volume
time                                                                   
2017-10-24 00:00:00-04:00   969.00   979.85   965.00   976.00   1744834
2017-10-25 00:00:00-04:00   978.00   984.44   966.24   972.91   2001000
2017-10-26 00:00:00-04:00   980.33   982.90   968.55   972.54   2373669
2017-10-27 00:00:00-04:00  1057.03  1105.58  1050.55  1100.99  11137045
2017-10-30 00:00:00-04:00  1095.42  1122.79  1093.56  1110.85   4459741

In [27]:
ba_data = api.get_barset('BA',timeframe='1D', limit=1000,).df
ba_data.head()

BA                                     
                             open      high     low     close   volume
time                                                                  
2017-10-24 00:00:00-04:00  263.42  267.2100  263.24  266.0500  3316040
2017-10-25 00:00:00-04:00  264.78  266.0000  254.50  258.4199  6031790
2017-10-26 00:00:00-04:00  259.72  261.4400  257.31  259.2700  2631747
2017-10-27 00:00:00-04:00  258.26  259.4994  255.80  256.3100  1955905
2017-10-30 00:00:00-04:00  255.45  260.4100  255.29  259.2400  2151914

In [28]:
nvda_data = api.get_barset('NVDA',timeframe='1D', limit=1000,).df
nvda_data.head()

NVDA                                    
                             open      high     low   close    volume
time                                                                 
2017-10-24 00:00:00-04:00  196.99  198.7400  195.94  198.67   8679881
2017-10-25 00:00:00-04:00  197.08  199.2200  191.17  193.64  17900060
2017-10-26 00:00:00-04:00  194.91  196.1600  193.81  195.59   6727585
2017-10-27 00:00:00-04:00  197.79  201.8700  196.75  201.75  12112237
2017-10-30 00:00:00-04:00  201.86  206.0899  201.22  203.76  10639517

In [29]:
brk_data = api.get_barset('BRK.A',timeframe='1D', limit=1000,).df
brk_data.head()

BRK.A                                      
                               open      high        low     close volume
time                                                                     
2017-09-29 00:00:00-04:00  274900.0  274900.0  273755.00  274900.0    198
2017-10-02 00:00:00-04:00  274999.0  278010.0  274500.00  278010.0    225
2017-10-03 00:00:00-04:00  278200.0  279660.0  277000.00  278630.0    214
2017-10-04 00:00:00-04:00  279200.0  279960.0  278384.84  278670.0    147
2017-10-05 00:00:00-04:00  279350.0  282000.0  278780.00  280360.0    305

In [30]:
gs_data = api.get_barset('GS',timeframe='1D', limit=1000,).df
gs_data.head()

GS                                 
                             open    high     low   close   volume
time                                                              
2017-10-24 00:00:00-04:00  243.60  246.25  243.00  244.87  1503543
2017-10-25 00:00:00-04:00  246.16  246.50  240.80  241.71  2072140
2017-10-26 00:00:00-04:00  242.33  244.00  241.53  241.72  1430800
2017-10-27 00:00:00-04:00  240.85  242.44  239.67  241.77  1354379
2017-10-30 00:00:00-04:00  240.10  242.59  239.53  240.86  1324530

In [31]:
cop_data = api.get_barset('COP',timeframe='1D', limit=1000,).df
cop_data.head()

COP                                
                            open   high     low   close   volume
time                                                            
2017-10-24 00:00:00-04:00  50.48  51.32  50.420  50.995  6532059
2017-10-25 00:00:00-04:00  51.06  51.13  49.750  49.960  5799954
2017-10-26 00:00:00-04:00  50.39  51.77  49.790  51.450  6276649
2017-10-27 00:00:00-04:00  51.16  52.23  50.200  51.210  6605795
2017-10-30 00:00:00-04:00  51.39  51.80  50.885  51.380  5274061

In [32]:
gild_data = api.get_barset('GILD',timeframe='1D', limit=1000,).df
gild_data.head()

GILD                               
                            open   high    low  close    volume
time                                                           
2017-10-24 00:00:00-04:00  80.18  80.38  78.80  80.09   5695051
2017-10-25 00:00:00-04:00  80.08  80.08  78.69  79.90   6835318
2017-10-26 00:00:00-04:00  78.44  79.50  77.00  77.84   9619233
2017-10-27 00:00:00-04:00  75.01  77.20  72.76  77.05  18999697
2017-10-30 00:00:00-04:00  76.10  76.93  75.25  75.98   9373807

In [33]:
ccoi_data = api.get_barset('CCOI',timeframe='1D', limit=1000,).df
ccoi_data.head()

CCOI                             
                            open  high     low  close  volume
time                                                         
2017-10-20 00:00:00-04:00  48.89  50.5  47.880  49.65  204367
2017-10-23 00:00:00-04:00  49.85  50.2  49.025  49.30  142925
2017-10-24 00:00:00-04:00  49.65  50.1  49.200  49.90  158656
2017-10-25 00:00:00-04:00  50.15  51.0  49.180  49.33  190010
2017-10-26 00:00:00-04:00  49.42  51.6  49.130  51.45  138453

In [34]:
proc_data = api.get_barset('PG',timeframe='1D', limit=1000,).df
proc_data.head()

PG                              
                            open   high    low  close   volume
time                                                          
2017-10-24 00:00:00-04:00  87.37  87.42  86.78  87.00  7370493
2017-10-25 00:00:00-04:00  86.90  87.11  86.28  86.83  6587502
2017-10-26 00:00:00-04:00  87.19  87.77  87.10  87.49  4597174
2017-10-27 00:00:00-04:00  87.53  87.53  86.45  87.03  5698218
2017-10-30 00:00:00-04:00  86.91  87.08  86.16  86.28  4220729

In [35]:
# aapl_data.dropna(inplace=True)
# amzn_data.dropna(inplace=True)
# ba_data.dropna(inplace=True)
# nvda_data.dropna(inplace=True)
# brk_data.dropna(inplace=True)
# gs_data.dropna(inplace=True)
# cop_data.dropna(inplace=True) 
# gild_data.dropna(inplace=True)
# ccoi_data.dropna(inplace=True)
# proc_data.dropna(inplace=True)

In [36]:
merged_stock_df= pd.concat([
    aapl_data, amzn_data, ba_data, nvda_data, brk_data,
    gs_data, cop_data, gild_data, ccoi_data, proc_data], axis='columns', join='inner')
#merged_stock_df = pd.concat(to_merge_dict.values())
#merged_stock_df.dropna(inplace=True)
merged_stock_df.head()

AAPL                                      \
                             open      high     low   close    volume   
time                                                                    
2017-10-24 00:00:00-04:00  156.29  157.4200  156.20  157.09  14016780   
2017-10-25 00:00:00-04:00  156.91  157.5500  155.27  156.41  16016689   
2017-10-26 00:00:00-04:00  157.23  157.8295  156.78  157.42  12374254   
2017-10-27 00:00:00-04:00  159.29  163.6000  158.70  163.03  36735155   
2017-10-30 00:00:00-04:00  163.89  168.0700  163.72  166.72  37107049   

                              AMZN                                       ...  \
                              open     high      low    close    volume  ...   
time                                                                     ...   
2017-10-24 00:00:00-04:00   969.00   979.85   965.00   976.00   1744834  ...   
2017-10-25 00:00:00-04:00   978.00   984.44   966.24   972.91   2001000  ...   
2017-10-26 00:00:00-04:00   980.33   982.90   968.55   972.54   2373669  ...   
2017-10-27 00:00:00-04:00  1057.03  1105.58  1050.55  1100.99  11137045  ...   
2017-10-30 00:00:00-04:00  1095.42  1122.79  1093.56  1110.85   4459741  ...   

                            CCOI                                   PG         \
                            open   high     low  close  volume   open   high   
time                                                                           
2017-10-24 00:00:00-04:00  49.65  50.10  49.200  49.90  158656  87.37  87.42   
2017-10-25 00:00:00-04:00  50.15  51.00  49.180  49.33  190010  86.90  87.11   
2017-10-26 00:00:00-04:00  49.42  51.60  49.130  51.45  138453  87.19  87.77   
2017-10-27 00:00:00-04:00  51.45  52.75  51.450  52.75  218586  87.53  87.53   
2017-10-30 00:00:00-04:00  52.55  52.55  51.675  52.40  291054  86.91  87.08   

                                                  
                             low  close   volume  
time                                              
2017-10-24 00:00:00-04:00  86.78  87.00  7370493  
2017-10-25 00:00:00-04:00  86.28  86.83  6587502  
2017-10-26 00:00:00-04:00  87.10  87.49  4597174  
2017-10-27 00:00:00-04:00  86.45  87.03  5698218  
2017-10-30 00:00:00-04:00  86.16  86.28  4220729  

[5 rows x 50 columns]

In [37]:
merged_stock_df.to_json('./Resources/stock_data.json')

In [38]:
#Bond Data
#iSHares 20 year plus treasury
bond_data = api.get_barset(['TLT', 'VOOG', 'VGLT', 'VMBS', 'IWF', 'IGSB', 'GOVT'],timeframe='1D', limit=1000,).df


In [39]:
bond_data.dropna(inplace=True)
bond_data.head()

GOVT                                    IGSB  \
                            open    high    low  close    volume    open   
time                                                                       
2017-10-24 00:00:00-04:00  25.11  25.135  25.10  25.12  636516.0  52.580   
2017-10-25 00:00:00-04:00  25.07  25.100  25.05  25.09  575142.0  52.600   
2017-10-26 00:00:00-04:00  25.09  25.110  25.05  25.06  758379.0  52.585   
2017-10-27 00:00:00-04:00  25.07  25.120  25.07  25.12  813036.0  52.565   
2017-10-30 00:00:00-04:00  25.16  25.190  25.13  25.18  530322.0  52.635   

                                                              ...   VMBS  \
                             high     low   close     volume  ...   open   
time                                                          ...          
2017-10-24 00:00:00-04:00  52.625  52.580  52.610  1359706.0  ...  52.67   
2017-10-25 00:00:00-04:00  52.605  52.570  52.595   413946.0  ...  52.56   
2017-10-26 00:00:00-04:00  52.605  52.570  52.588   534592.0  ...  52.64   
2017-10-27 00:00:00-04:00  52.630  52.565  52.605   446882.0  ...  52.56   
2017-10-30 00:00:00-04:00  52.640  52.605  52.613   789298.0  ...  52.72   

                                                              VOOG            \
                              high    low  close    volume    open      high   
time                                                                           
2017-10-24 00:00:00-04:00  52.6900  52.61  52.62  326018.0  131.59  131.5900   
2017-10-25 00:00:00-04:00  52.8600  52.53  52.57  404450.0  131.26  131.4175   
2017-10-26 00:00:00-04:00  52.6411  52.52  52.52  255541.0  131.04  131.2200   
2017-10-27 00:00:00-04:00  52.6400  52.51  52.62  318618.0  132.04  133.1400   
2017-10-30 00:00:00-04:00  52.7600  52.69  52.75  311670.0  132.99  133.1500   

                                                      
                                low   close   volume  
time                                                  
2017-10-24 00:00:00-04:00  131.2099  131.41  39516.0  
2017-10-25 00:00:00-04:00  130.2600  130.94  35633.0  
2017-10-26 00:00:00-04:00  130.8400  131.01  27887.0  
2017-10-27 00:00:00-04:00  132.0400  132.97  40996.0  
2017-10-30 00:00:00-04:00  132.4410  132.86  36904.0  

[5 rows x 35 columns]

In [40]:
bond_data.to_json('./Resources/bond_data.json')

In [41]:
#Machine Learning for Investments
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Needed for decision tree visualization
import pydotplus
from IPython.display import Image

In [42]:
#Calcualtions for Daily Return which help with creating a target for ML
merged_stock_df.loc[:, ('AAPL', 'Daily_Returns')] = merged_stock_df['AAPL']['close'].pct_change()
merged_stock_df.loc[:,('AMZN','Daily_Returns')] = merged_stock_df['AMZN']['close'].pct_change()
merged_stock_df.loc[:,('BA','Daily_Returns')] = merged_stock_df['BA']['close'].pct_change()
merged_stock_df.loc[:,('NVDA','Daily_Returns')] = merged_stock_df['NVDA']['close'].pct_change()
merged_stock_df.loc[:,('BRK.A','Daily_Returns')] = merged_stock_df['BRK.A']['close'].pct_change()
merged_stock_df.loc[:,('GS','Daily_Returns')] = merged_stock_df['GS']['close'].pct_change()
merged_stock_df.loc[:,('COP','Daily_Returns')] = merged_stock_df['COP']['close'].pct_change()
merged_stock_df.loc[:,('GILD','Daily_Returns')] = merged_stock_df['GILD']['close'].pct_change()
merged_stock_df.loc[:,('CCOI','Daily_Returns')] = merged_stock_df['CCOI']['close'].pct_change()
merged_stock_df.loc[:,('PG','Daily_Returns')] =merged_stock_df['PG']['close'].pct_change()
merged_stock_return_df=  merged_stock_df.copy()
merged_stock_return_df.dropna(inplace=True)

In [43]:
merged_stock_return_df.loc[:, ('AAPL', 'Daily_Returns')] = merged_stock_df['AAPL']['close'].pct_change()
merged_stock_return_df.loc[:,('AMZN','Daily_Returns')] = merged_stock_df['AMZN']['close'].pct_change()
merged_stock_return_df.loc[:,('BA','Daily_Returns')] = merged_stock_df['BA']['close'].pct_change()
merged_stock_return_df.loc[:,('NVDA','Daily_Returns')] = merged_stock_df['NVDA']['close'].pct_change()
merged_stock_return_df.loc[:,('BRK.A','Daily_Returns')] = merged_stock_df['BRK.A']['close'].pct_change()
merged_stock_return_df.loc[:,('GS','Daily_Returns')] = merged_stock_df['GS']['close'].pct_change()
merged_stock_return_df.loc[:,('COP','Daily_Returns')] = merged_stock_df['COP']['close'].pct_change()
merged_stock_return_df.loc[:,('GILD','Daily_Returns')] = merged_stock_df['GILD']['close'].pct_change()
merged_stock_return_df.loc[:,('CCOI','Daily_Returns')] = merged_stock_df['CCOI']['close'].pct_change()
merged_stock_return_df.loc[:,('PG','Daily_Returns')] = merged_stock_df['PG']['close'].pct_change()

merged_stock_return_df.dropna(inplace=True)


In [44]:
# l = []
# for x in dict(merged_stock_df.columns):
#     l.append((x, 'close'))
# new_merged_stock_df = merged_stock_df[l]
#new_merged_stock_df.head()

In [45]:
zero_risk_df = pd.read_json('zero_risk_portfolio.json', orient='columns')

# zero_risk_df.rename(columns=
#     'GOVT_open','GOVT_high','GOVT_low', 'GOVT_close',	
#     'GOVT_volume',	'IGSB_open', 'IGSB_high',	'IGSB_low',
#     'IGSB_close',	'IGSB_volume', 'IWF_open',	'IWF_high',	
#     'IWF_low',	'IWF_close',	'IWF_volume', 'TLT_open',	'TLT_high',	
#     'TLT_low',	'TLT_close',	'TLT_volume', 'VGLT_open',	'VGLT_high',	
#     'VGLT_low',	'VGLT_close',	'VGLT_volume',	'VMBS_open',	'VMBS_high',	
#     'VMBS_low',	'VMBS_close',	'VMBS_volume',	'VOOG_open',	
#     'VOOG_high',	'VOOG_low',	'VOOG_close', 'VOOG_volume')

#zero_risk_df.loc[['(GOVT, open)']]

In [46]:
## Imports for Neural Network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import tensorflow as tf


In [47]:
ba_data_modeling = ba_data['BA']
ba_data_modeling['daily_returns']= ba_data_modeling['close'].pct_change()
ba_data_modeling['profit']= ""
ba_data_modeling.head()

<ipython-input-47-30608d341b1b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ba_data_modeling['daily_returns']= ba_data_modeling['close'].pct_change()
<ipython-input-47-30608d341b1b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ba_data_modeling['profit']= ""


,open,high,low,close,volume,daily_returns,profit
time,,,,,,,
2017-10-24 00:00:00-04:00,263.42,267.2100,263.24,266.0500,3316040,NaN,
2017-10-25 00:00:00-04:00,264.78,266.0000,254.50,258.4199,6031790,-0.028679,
2017-10-26 00:00:00-04:00,259.72,261.4400,257.31,259.2700,2631747,0.003290,
2017-10-27 00:00:00-04:00,258.26,259.4994,255.80,256.3100,1955905,-0.011417,
2017-10-30 00:00:00-04:00,255.45,260.4100,255.29,259.2400,2151914,0.011431,


In [48]:
profit =[] 
for x in ba_data_modeling['daily_returns']:
    if x > 0:
        profit.append('gain')
    else:
        profit.append('loss')
ba_data_modeling['profit']= profit
ba_data_modeling.head()

<ipython-input-48-9e1b18e61ae4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ba_data_modeling['profit']= profit


,open,high,low,close,volume,daily_returns,profit
time,,,,,,,
2017-10-24 00:00:00-04:00,263.42,267.2100,263.24,266.0500,3316040,NaN,loss
2017-10-25 00:00:00-04:00,264.78,266.0000,254.50,258.4199,6031790,-0.028679,loss
2017-10-26 00:00:00-04:00,259.72,261.4400,257.31,259.2700,2631747,0.003290,gain
2017-10-27 00:00:00-04:00,258.26,259.4994,255.80,256.3100,1955905,-0.011417,loss
2017-10-30 00:00:00-04:00,255.45,260.4100,255.29,259.2400,2151914,0.011431,gain


In [49]:
# enc_ba = pd.get_dummies(ba_data_modeling)
y= ba_data_modeling['profit']
X= ba_data_modeling.drop(columns='profit')

In [50]:
X.head()

,open,high,low,close,volume,daily_returns
time,,,,,,
2017-10-24 00:00:00-04:00,263.42,267.2100,263.24,266.0500,3316040,NaN
2017-10-25 00:00:00-04:00,264.78,266.0000,254.50,258.4199,6031790,-0.028679
2017-10-26 00:00:00-04:00,259.72,261.4400,257.31,259.2700,2631747,0.003290
2017-10-27 00:00:00-04:00,258.26,259.4994,255.80,256.3100,1955905,-0.011417
2017-10-30 00:00:00-04:00,255.45,260.4100,255.29,259.2400,2151914,0.011431


In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

In [52]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [53]:
neuron = Sequential()

In [54]:
"""Multi Layer for Neural Network, we will have to constantly tweek these numbers.
For now we will use Three Layers and determine the activations as well"""
number_inputs = 6
number_hidden_nodes = 3

neuron.add(Dense(input_dim=number_inputs, units=number_hidden_nodes, activation="relu"))

#Second Layer
second_number_inputs = 8
second_number_hidden_nodes = 6

neuron.add(Dense(input_dim=number_inputs, units=second_number_hidden_nodes, activation="relu"))

#Third Layer
#Second Layer
third_number_inputs = 8
third_number_hidden_nodes = 6

neuron.add(Dense(input_dim=number_inputs, units=third_number_hidden_nodes, activation="relu"))


In [67]:
outputs = 5
neuron.add(Dense(units=outputs, activation="sigmoid"))


In [68]:
neuron.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 21        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 24        
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 35        
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 30        
Total params: 182
Trainable params: 182
Non-trainable params: 0
__________________________________________________________

In [69]:
#labels = y_train.unique()
#y_train = tf.one_hot(y_train, depth=len(labels))
#neuron.add(Dense(len(labels), activation='softmax'))

In [70]:
neuron.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [71]:
model = neuron.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150


ValueError: in user code:

    C:\Users\bowsk\anaconda3\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\bowsk\anaconda3\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\bowsk\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\bowsk\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\bowsk\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\bowsk\anaconda3\lib\site-packages\keras\engine\training.py:835 run_step  **
        outputs = model.train_step(data)
    C:\Users\b

In [ ]:
# new_merged_stock_df = merged_stock_df[l]
#     if x > 0:
#         print("profit")
#     else:
#         print("loss")

In [ ]:
#iShares 1-5 year Investment grade bond
# igsb_data = api.get_barset('IGSB',timeframe='1D', limit=1000,).df
# igsb_data.head()

In [ ]:
#iShares US Treasury Bond ETF
# govt_data = api.get_barset('GOVT',timeframe='1D', limit=1000,).df
# govt_data.head()